# Importando as Bibliotecas

In [ ]:
df_bow = pd.DataFrame( titulo_bow , columns = cv.get_feature_names() )

feature_vector_1 = df_bow.loc[0, :]
feature_vector_2 = df_bow.loc[1, :]

calcular_dis_cos(feature_vector_1, feature_vector_2)

In [ ]:
indices_1, valores_1 = calcular_k_maiores(resultado, 1)
indices_10, valores_10 = calcular_k_maiores(resultado, 10)
indices_50, valores_50 = calcular_k_maiores(resultado, 50)

for indices in [indices_1, indices_10, indices_50]:

    # criando uma coluna nova no df
    df_pp[f'k-{len(indices[0])}'] = 0

    for i in range(len(indices)):

        for j in range(len(indices[0])):
            
            if df_pp['ean'].loc[i] == df_pp['ean'].loc[indices[i][j]]:

                df_pp[f'k-{len(indices[0])}'].loc[i] = 1

In [ ]:
def calcular_k_maiores(res, k):

    indices = []
    valores = []

    for i in range(len(res)):
        
        ind = res[i].argsort()[-k:]
        val = res[i][res[i].argsort()[-k:]]

        indices.append( list(reversed(ind)) )
        valores.append( list(reversed(val)) )

    return indices, valores

In [ ]:
def criar_nao_pares_hn(df, n_matches, ean_repetido, indices_resultado, colunas = COLUNAS):

    lista_3 = []
    df_nao_pares_hn = pd.DataFrame(columns = colunas)

    limite = ceil(n_matches/len(ean_repetido))
    print(f"limite: {limite}")

    for i in range(len(ean_repetido)):

        lista_1 = []
        lista_2 = []

        ean = ean_repetido[i]
        indice_df = df[df['ean'] == ean].head(1).index.values[0] # me da o 1º indice do dataframe onde tem o EAN repetido
        lista_1.append(indice_df)

        flag_encontrou = False
        j = 0
        cont = 0
        while (flag_encontrou == False):
            
            indice_rank = indices_resultado[indice_df][j]

            if df['ean'].loc[indice_df] != df['ean'].loc[indice_rank]:


                lista_2.append(indice_rank)

                cont += 1

                if cont > limite:
                    flag_encontrou = True

            j += 1

        lista_3.append([ean, lista_1, lista_2])

    return lista_3

In [2]:
import tensorflow_hub as hub
import numpy as np
import tensorflow_text

In [3]:
# Some texts of different lengths.
english_sentences = ["dog", "Puppies are nice.", "I enjoy taking long walks along the beach with my dog."]
italian_sentences = ["cane", "I cuccioli sono carini.", "Mi piace fare lunghe passeggiate lungo la spiaggia con il mio cane."]
japanese_sentences = ["犬", "子犬はいいです", "私は犬と一緒にビーチを散歩するのが好きです"]
portuguese_sentences = ["Olá amigo, tudo bem?"]

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

# Compute embeddings.
en_result = embed(english_sentences)
it_result = embed(italian_sentences)
ja_result = embed(japanese_sentences)
br_result = embed(portuguese_sentences)

# Compute similarity matrix. Higher score indicates greater similarity.
similarity_matrix_it = np.inner(en_result, it_result)
similarity_matrix_ja = np.inner(en_result, ja_result)

In [1]:
from sentence_transformers import SentenceTransformer

Semantic Similarity

These models find semantically similar sentences within one language or across languages:

distiluse-base-multilingual-cased-v1: Multilingual knowledge distilled version of multilingual Universal Sentence Encoder. Supports 15 languages: Arabic, Chinese, Dutch, English, French, German, Italian, Korean, Polish, Portuguese, Russian, Spanish, Turkish.

distiluse-base-multilingual-cased-v2: Multilingual knowledge distilled version of multilingual Universal Sentence Encoder. This version supports 50+ languages, but performs a bit weaker than the v1 model.

paraphrase-multilingual-MiniLM-L12-v2 - Multilingual version of paraphrase-MiniLM-L12-v2, trained on parallel data for 50+ languages.

paraphrase-multilingual-mpnet-base-v2 - Multilingual version of paraphrase-mpnet-base-v2, trained on parallel data for 50+ languages.

In [21]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

query_embedding = model.encode('How many people live in London?')

#The passages are encoded as [ [title1, text1], [title2, text2], ...]
passage_embedding = model.encode([['London', 'London has 9,787,426 inhabitants at the 2011 census.']])

print("Similarity:", util.cos_sim(query_embedding, passage_embedding))

Similarity: tensor([[0.6691]])


Fazendo o fit do SentenceTransformer usando mais parâmetros

In [ ]:
def pipeline_st(df_treino):

    num_epocas = 3
    
    # carregando o modelo
    modelo = SentenceTransformer('distiluse-base-multilingual-cased-v1')

    dados_treino = df_treino.apply(lambda row: retornar_input_example(row['titulo_1'], row['titulo_2'], float(row['match'])), axis = 1)

    treino_dataloader = DataLoader(dados_treino, shuffle = True, batch_size = 1)
    treino_perda = losses.CosineSimilarityLoss(model = modelo)

    avaliador = evaluation.EmbeddingSimilarityEvaluator(df_treino['titulo_1'], df_treino['titulo_2'], df_treino["match"])

    # fine-tune do mocelo
    modelo.fit(
               train_objectives=[(treino_dataloader, treino_perda)],
               evaluator = avaliador,
               epochs = num_epocas,
               evaluation_steps = 1000,
               warmup_steps = 100,
               output_path = CAMINHO_MODELO
              )
    
    #Predict test data
    '''label_pred = get_test_prediction(model, ds_test)

    return (name, bert_history, y_test, label_pred)'''

    # metrics = calc_metrics(y_test, result, name)
    # return metrics

Inserindo a coluna "categoria" nos pares

In [ ]:
lista_nomes = ["celulares", "notebooks", "geladeiras", "fogoes", "tvs"]
df = pd.read_csv(f"Dados/Pares/Sem Categoria/pares_{lista_nomes[0]}.csv")
df.insert(loc = 12, column = 'categoria', value = lista_nomes[0])
df.to_csv(f"Dados/Pares/pares_{lista_nomes[0]}.csv", index = False)